In [29]:
#helper functions
import math
from sympy.ntheory import factorint
import numpy as np
from progressbar import *   
from tqdm import tqdm
import random

def gcd(x, y): 
    while(y): 
        x, y = y, x % y 
    return x

def prime(x): #TODO - improve
    #if 
    for i in range(2, round(math.sqrt(x)) + 1):
        if x % i == 0:
            return False
    return True


def add_cols(M, a, b):
    
    return np.array([(M[i][a] + M[i][b]) % 2 for i in range(len(M))])

def find_pivot(M, col):
    for i, row in enumerate(range(len(M))):
        if M[row][col] == 1: return i
    
    return None


def reduce_matrix(M):
    #taken from https://www.sciencedirect.com/science/article/pii/074373159190115P
    #Assume M is K+1 x K
    
    ##Gaussian elimination of matrix
    K = len(M[0])
    
    marked = [0] * (K+1)
    pivots = [None] * K
    
    

    for j in tqdm(range(K)):
        i = find_pivot(M, j)
        if i is not None:
            marked[i] = True
            pivots[j] = i
            for k in range(K):
                if M[i][k] == 1 and j != k:
                    #print("k=", k)
                    M[:, k] = add_cols(M, j, k)
    
    #finding dependent rows
    rows = []
    for i in tqdm(range(K+1)):
        if not marked[i]:
            indices = [i]
            for j in range(K):
                if M[i][j] == 1:
                    indices.append(pivots[j])
            rows.append(indices)
                    
    return M, rows


def mod2(xs):
    return np.array([x % 2 for x in xs])


def find_a(a, p):
    """
    Find x such that x^2 = a (mod p)
    """
    t = 0
    while jacobi(t**2 - a, p) != -1:
        t = random.randint(0, p-1)
    
    i = (t**2 - a) % p**2
    j = (t + math.sqrt(i)) % p**2
    x = j**(p+1)/2 % p**2
    
    return x 


def jacobi(a, m):
    """return jacobi(a/m)"""
    a = a % m 
    t = 1
    while a != 0:
        while a % 2 == 0:
            a /= 2
            if m % 8 in [3, 5]:
                t = -t
        
        a, m = m, a
        
        if a % 4 == 3 and m % 4 == 3:
            t = -t
        
        a = a % m
        
    if m == 1: return t
    
    return 0
    

                
                

In [30]:
find_a(3, 11)

-62**2 % 11

6

In [33]:
def quadratic_sieve(n, verbose=False):
    
    B = math.ceil(math.exp(math.sqrt(np.log(n) * np.log(np.log(n)))))
    
    if verbose: print("B =", B)
    

    
    smooths = []
    
    S = []
    
    print("finding primes...")
    b_primes = [i for i in range(2, B + 1) if prime(i)]
    K = len(b_primes)
    
    print("finding a_i's....")
    #A1 - find ± a_i where a_i^2 = n (mod p_i)
    a = []
    for p in tqdm(b_primes[1:]):
        a.append(find_a(n % p, p))
    
    
    
    
    ## Trying different x values
    
    print("sieving")
    
    x = round(math.sqrt(n))
    
    widgets = ['b-smooth count: ', Percentage(), ' ', Bar(marker='x',left='[',right=']'),
           ' ', ETA(), ' ']

    pbar = ProgressBar(widgets=widgets, maxval=K+1)
    pbar.start()
    while len(smooths) <= K:
        y = (x**2) % n
        factors = factorint(y)
        if max(factors.keys()) <= B: #is b-smooth
            
            exps = np.zeros(len(b_primes))
            for f in factors:
                exps[b_primes.index(f)] = factors[f] 
                
            if verbose: print("x =", x, ": ", exps)
            S.append((x, x**2 - n))
            smooths.append(exps)
        x += 1
        pbar.update(len(smooths)) #this adds a little symbol at each iteration
    
    pbar.finish()
    
    ## Gaussian elimination ....
    print("matrixing")
    M = np.array([mod2(s) for s in smooths])
    
    reduced_M, rows = reduce_matrix(M)
    
    #rows[0] contains the indices of M that sum to zero
    
    print("solving for x,y...")
    
    x = 1
    for r in rows[0]:
        x = (x * S[r][0]) % n
    
    y = 1 #how to use a_i's??
    for j in range(K):
        p = sum([smooths[r][j] for r in rows[0]])/2
        y = (y * b_primes[j] ** p) % n
    y = y % n
    
    d = gcd(x-y, n)
    
    print(n, "=", d, "*", n/d)
    assert n % d == 0
    
    return d
        
        
    
    
quadratic_sieve(16921456439215439701, verbose=False)
#16921456439215439701

finding primes...


  0%|          | 33/35596 [00:00<00:00, 42852.02it/s]

finding a_i's....


OverflowError: (34, 'Result too large')